# Exam Data Science I  
### Exam I in the Summer Term 2024

## General Information

* You have one week to complete the exam. 

* You can use all sources freely, but you must name them correctly. If you use ChatGPT or similar software, you must include the prompt.

* You should use the following packages: `numpy, pandas, scipy, geopy, scikit-learn/sklearn, matplotlib, seaborn, openpyxl` and Python's native libraries. These are sufficient to solve the exam. If you use other libraries, justify their use.

* Code must be sufficiently commented to be understandable. Write functions whenever you reuse code. In general, follow the guidelines from the lecture. Points may be deducted due to poorly structured or incomprehensible code.

* **Always justify (!)** decisions regarding the choice of plots, hypothesis tests, etc. in writing and **interpret** your results.

* You are **not** allowed to seek help or advice from other people in any way. 

* Please submit the complete repository as a `.zip` file with the name `surname_matrikelnummer.zip` by August 8th, 2024 at 12:00 noon on StudIP to the folder `Submission - Exam 1`.

* Also add the signed code of conduct to the `.zip` file. 

* If you have any questions, please contact us via Rocketchat in a timely manner.

In [1]:
# IMPORT LIBRARIES
import pandas as pd

## Exercises and Points:

<table>
  <thead>
    <tr>
      <th colspan="3">Exercise 1 - Data Preprocessing</th>
      <th colspan="2">Exercise 2 - Plotting</th>
      <th colspan="2">Exercise 3 - Statistics</th>
      <th colspan="2">Exercise 4 - Machine Learning </th>
    </tr>
    <tr>
      <th>Exercise 1.1</th>
      <th>Exercise 1.2</th>
      <th>Exercise 1.3</th>
      <th>Exercise 2.1</th>
      <th>Exercise 2.2</th>
      <th>Exercise 3.1</th>
      <th>Exercise 3.2</th>
      <th>Exercise 4.1</th>
      <th>Exercise 4.2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10 points </td>
      <td>12 points </td>
      <td>2 points </td>
      <td>11 points</td>
      <td>27 points </td>
      <td>13 points </td>
      <td>5 points </td>
      <td>10 points </td>
      <td>10 points </td>
    </tr>
    <!-- Add more rows as needed -->
  </tbody>
</table>


_____
## Exercise 0: Setup

The exam folder contains a `Dockerfile` in which all relevant libraries are defined. The `Dockerfile` builds up on the Jupyter Server image. Use this Dockerfile to first create a Docker image and then start a Docker container from that image. Afterwards log into the Jupyter Server instance to work on the exam. We strongly recommend using the Docker environment to avoid version conflicts between the various libraries. Code that does not run in this environment will be graded as **non functional**.

____
## Exercise 1: Data Preprocessing (24 points)

### Data Description

In the data folder, you will find the parking data in Göttingen city for the year 2023 as one file per month. The parking tickets bought using the stationary ticket machines are in the files whose names start with `Cale` and the parking tickets bought with the parkster app are in the files whose names start with `Parkster`. <br> 
The file `parkzone_latlong.csv` provides further geographical information regarding the parkzones and the file `psa_latlong.csv` provides geographical information about the parking machines within the parkzones.

The parking data provided is genuine raw data and comes directly from the city of Göttingen. We have only added the geographical information. 

*Please note:*
- *in the following the term "parking ticket" refers to a piece of paper that entitles you to park and not a fine for illegal parking.*
- *although we only have data from February to December, we will refer to all data available as yearly.*
- *due to the size of the data it might be necessary to use your memory efficiently, therefore avoid storing several copies of the same DataFrame.*

#### Exercise 1.1 - Data Loading (10 points)
Load the files for the machines (`Cale-*`) and for the app (`Parkster-*`) and concatenate all months to get two DataFrames, one for the machine and one for the app purchase for the whole year.
Also load the machine (`psa_latlong.csv`) and the parkzone information (`parkzones_latlong.csv`).

You will find the values `0` and `999` in the `Automaten ID` column for the machine data. Change the `0`s to `1`s and drop all entries with `999`.
Also check whether there are any duplicate rows and delete them. 

#### Exercise 1.2 - Merging and Formating (12 points)
Concatenate the app and machine data, using the parkzones *(in `parkzones_latlong.csv`)* and the parking machine number *(in `psa_latlong.csv`)*. Make sure that you have the geographical information for both, the parking machines and the parkzones, in the final data frame. 
Use the columns `Kaufdatum Lokal` and `Start` for the date of purchase, encode the column as a `datetime` object and use it as index column. Also make sure that the other columns have a reasonable data format. 


*Note: Having `NaN` values for some columns in the rows belonging to app purchases can be expected.*

#### Exercise 1.3 - DataFrame Check (2 points)
The cleaned and complete DataFrame for the following exercises should look like `data/clean_dataframe.csv`, which can be read in correctly as follows:

In [ ]:
pd.read_csv('data/clean_dataframe.csv', parse_dates=['time'], index_col='time', dtype={'machine_ID': 'Int64', 
                                                                           'fee': 'float64', 
                                                                           'category': 'object', 
                                                                           'street': 'object', 
                                                                           'latitude_machine': 'float64', 
                                                                           'longitude_machine': 'float64', 
                                                                           'zone': 'int64', 
                                                                           'latitude_zone': 'float64', 
                                                                           'longitude_zone': 'float64'}).sort_index()

Make sure that your DataFrame aligns with `clean_dataframe.csv`. To prove this, use the function [`pandas.DataFrame.equals`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.equals.html).

If `pandas.DataFrame.equals` does not return `True`, please continue working with `clean_dataframe.csv` and indicate this using a Markdown cell. In that case you won't get any points for this subtask.

_____
## Exercise 2: Plotting (38 points)



### Exercise 2.1 - Analysis of Parking Machines (11 points)
The city of Göttingen would like to have an overview of the returns of the **individual parking machines** and hires you to carry out an initial explorative analysis of the sales volume and the geographical location of the machines.


#### 2.1.1 (6 points)
Find the top 5 parking machines regarding their sales volume in 2023 and plot their weekly turnover over the year.

#### 2.1.2 (5 points)
The location of the machines might also have an influence on the turnover. 

Familiarize yourself with the function `plot_map` from the library `dsplotter`. Use the function to plot the yearly turnover for each location. Make the color **and** the radius of the location marker dependent on the yearly turnover. Can you see any relationship between the machines with a high yearly turnover? 

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dsplotter import plot_map


def load_and_prepare_data(clean_df):

    machine_sales = clean_df[clean_df["category"] == "machine"]
    top_5_by_sales = (
        machine_sales.groupby("machine_ID")["fee"].sum().nlargest(5, "first")
    )

    weekly_sales = machine_sales[
        machine_sales["machine_ID"].isin(top_5_by_sales.index)
    ]
    weekly_sales = (
        weekly_sales.groupby([pd.Grouper(freq="W"), "machine_ID"])["fee"]
        .sum()
        .reset_index()
    )

    return weekly_sales, top_5_by_sales


def plot_heatmap(weekly_sales):
    pivot_data = weekly_sales.pivot(
        index="time", columns="machine_ID", values="fee"
    )

    # Add week numbers to the index
    pivot_data["week_number"] = pivot_data.index.isocalendar().week

    plt.figure(figsize=(20, 16))

    ax = sns.heatmap(
        pivot_data.iloc[:, :-1],
        cmap="YlOrRd",
        cbar_kws={"label": "Weekly Sales (EUR)"},
        annot=True,
        fmt=".2f",
        linewidths=0.5,
    )

    plt.xticks(rotation=0)

    # Set y-axis ticks to show every week number
    ax.set_yticks(range(len(pivot_data)))
    ax.set_yticklabels(pivot_data["week_number"], rotation=0)

    plt.title("Weekly Sales Heatmap for Top 5 Parking Machines (2023)")
    plt.xlabel("Machine ID")
    plt.ylabel("Week Number")
    plt.tight_layout()
    plt.savefig("top_5_machines_heatmap.png", dpi=300)
    plt.close()


def main():
    clean_df = pd.read_csv(
        "data/clean_dataframe.csv",
        parse_dates=["time"],
        index_col="time",
        dtype={
            "machine_ID": "Int64",
            "fee": "float64",
            "category": "object",
            "street": "object",
            "latitude_machine": "float64",
            "longitude_machine": "float64",
            "zone": "int64",
            "latitude_zone": "float64",
            "longitude_zone": "float64",
        },
    ).sort_index()

    # 2.1.1
    weekly_sales, top_5_by_sales = load_and_prepare_data(clean_df)

    print(weekly_sales.head(5))

    print("Top 5 machines by sales volume:")
    print(top_5_by_sales)

    plot_heatmap(weekly_sales)
    # /---
    # 2.1.2

    yearly_sales_by_machine: pd.DataFrame = (
        clean_df[clean_df["category"] == "machine"]
        .groupby("machine_ID")[
            ["fee", "latitude_machine", "longitude_machine"]
        ]
        .agg(
            {
                "fee": "sum",
                "latitude_machine": "first",
                "longitude_machine": "first",
            }
        )
    ).reset_index()  # reset_index() ensures that we get a DataFrame, which is only needed to silence the LSP (no functional difference)

    yearly_sales_by_machine = yearly_sales_by_machine.rename(
        columns={
            "fee": "yearly_sales_eur",
            "latitude_machine": "latitude",
            "longitude_machine": "longitude",
        }
    )

    yearly_sales_by_machine.info()

    print("\nYearly sales per machine and location:")
    print(yearly_sales_by_machine)

    plot_map(yearly_sales_by_machine, "yearly_sales_eur", "yearly_sales_eur")#, radius_scale=15)


if __name__ == "__main__":
    main()

        time  machine_ID    fee
0 2023-02-05           8  586.9
1 2023-02-05          17  547.6
2 2023-02-05          18  674.3
3 2023-02-05          53  745.9
4 2023-02-05          76  467.7
Top 5 machines by sales volume:
machine_ID
53    55854.3
17    42122.0
18    40313.5
8     39005.7
76    37815.5
Name: fee, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   machine_ID        196 non-null    Int64  
 1   yearly_sales_eur  196 non-null    float64
 2   latitude          196 non-null    float64
 3   longitude         196 non-null    float64
dtypes: Int64(1), float64(3)
memory usage: 6.4 KB

Yearly sales per machine and location:
     machine_ID  yearly_sales_eur   latitude  longitude
0             1           24061.9  51.529542   9.939956
1             2            8647.2  51.533104   9.926817
2             3          

### Exercise 2.2 - Analysis of Machine and App Usage per Zone (27 points)
As part of the city's digitization initiative, the Parkster app was introduced a few years ago as an alternative to parking machines. 
So far we have focused only on the parking machines and therefore discarded a bulk of ticket sales.
The city would like to carry out an initial visual analysis of the acceptance of the app in the individual parkzones in 2023 and analysis the total parking turnover. 

*Note that we can only analyse the total parking ticket sales by summarizing all parking machines per parkzone.*

#### 2.2.1 (6 points)
Do users prefer the app or the machine usage? 

Use an appropriate plot to show the average machine or app usage rate per **parkzone** for the whole year of 2023. What do you observe?

#### 2.2.2 (9 points)
How frequently are the individual parkzones used? 

Visualize the total number of parking tickets and the machine usage for each parkzone in 2023 in one plot. Use the a `log`-scale for the y-axis. What do you observe?

#### 2.2.3 (7 points)
The previous plot gives us an idea about the total number of *users* per parkzone. This number is likely highly correlated with the number of parking lots per zone. If we want to compare different parkzones with regard to their parking tickets sold, we should normalize them by the number of parking lots available. This way we can find out which zones are used most frequently. Because we don't have the number of parking lots available for each zone, we can only use the number of parking machines as a rough approximation for their sizes. 

Use the information from the `psa_latlong.csv` and reproduce the previous plot using the number of parking tickets normalized by the number of machines for each zone. Use the a `log`-scale for the y-axis.
Which parkzone is the busiest one?

#### 2.2.4 (5 points)
So far we haven't used the geographical information of the parkzones. 

Use the `plot_map`-function to plot the location (center) of all parkzones, their average tickets sold per machine and the rate of machine users. Color the location by the rate of machine users and set the radius using the tickets sold per machine. What do you observe?

___
## Exercise 3: Statistics (18 points)


#### Exercise 3.1 - t-Test (13 points)
In addition to our visual analysis, we now also want to carry out a statistical study of machine vs. app usage.

To do this, determine the daily rate of machine use per parkzone. Carry out a t-test **for each parkzone**, which statistically tests whether parkers prefer to use the app in the respective zone. Write down the corresponding pair of hypotheses, carry out the test and interpret your test results. Use a significance level of 0.05 for your test decision. Which fundamental assumption of the statistical test could be violated?

#### Exercise 3.2 - Statistical Reasoning (5 points)
Assume that for zone `37106` the rate of app users (and thus the rate of machine users) is `0.5`. 
The city sends you the data for 2024. 

How many days can you expect the app usage rate to be significantly greater than the machine usage rate at a significance level of 5%? Explain why that is the case. Assume that the behavior of the users has not changed compared to 2023.

___
## Exercise 4: Machine Learning (20 points)

Use the k-Nearest Neighbors (k-NN) algorithm to build a machine learning model that predicts the parking method (app or machine) based on location and fee as input features. Use parking data between fee value 2 euros and 7 euros.

#### Excersice 4.1 - Model Building and Hyperparameter Search (10 points)
Preprocess the data appropriately, perform hyperparameter search with cross validation for the optimal k value, visualize your optimal choice of k and finally use your optimal k-value to build the model.

**Hint:** Use 30% of all the data to speed up hyperparameter search and use the whole dataset for model building

#### Exercise 4.2 - Visualizing Model Predictions (10 points)
Create a `100 x 100` grid of longitude/latitude values using the min and max values from your dataset and visualize the predictions of the k-NN model **for three different parking fees** - 3, 5, and 7 euros, using the `plot_map` function. Set the color of the markers on the map to your prediction. Point out at least 2 visual changes in the predicted pattern of parking method across the city for these three different fee values.